Required initial steps:
    
    1. The text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt within the data_projects/sports_data folder.
    
    2. The file is then processed through the bash script file a_updater.sh to convert it to a csv: Type the following in Terminal:  ./a_updater.sh [from the data_projects/sports_data directory]
    
    3. Copy to afl_dataexploration subfolder. The bash file saves the processed file as newlist.csv, ready to be imported from within Jupyter as below.

In [1]:
import os

In [2]:
os.getcwd()

'/Users/stevegabriel/data_projects/sports_data/afl_dataexploration'

In [3]:
import pandas as pd
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

In [4]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [10]:
# Modify index to start at 1 
trans.index += 1
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state
15526,15525,27-Jun-2019,R15,Essendon,12,5,77,GW Sydney,10,11,71,Docklands,VIC
15527,15526,28-Jun-2019,R15,Geelong,14,12,96,Adelaide,10,9,69,Kardinia Park,VIC
15528,15527,29-Jun-2019,R15,Hawthorn,9,17,71,West Coast,11,11,77,M.C.G.,VIC
15529,15528,29-Jun-2019,R15,Sydney,14,9,93,Gold Coast,7,9,51,S.C.G.,NSW
15530,15529,29-Jun-2019,R15,Collingwood,5,7,37,North Melbourne,11,15,81,Docklands,VIC
15531,15530,29-Jun-2019,R15,Port Adelaide,5,11,41,Western Bulldogs,10,6,66,Adelaide Oval,SA
15532,15531,30-Jun-2019,R15,St Kilda,10,10,70,Richmond,16,7,103,Docklands,VIC
15533,15532,30-Jun-2019,R15,Brisbane Lions,15,17,107,Melbourne,11,8,74,Gabba,QLD
15534,15533,30-Jun-2019,R15,Fremantle,11,9,75,Carlton,11,13,79,Perth Stadium,WA


In [6]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition','Riverway Stadium']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [7]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [8]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13102
WA        656
SA        587
NSW       524
QLD       488
TAS        95
ACT        50
NT         25
INTL        6
Name: state, dtype: int64

### Check that the most recent round is included:

In [9]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state
15525,15525,27-Jun-2019,R15,Essendon,12,5,77,GW Sydney,10,11,71,Docklands,VIC
15526,15526,28-Jun-2019,R15,Geelong,14,12,96,Adelaide,10,9,69,Kardinia Park,VIC
15527,15527,29-Jun-2019,R15,Hawthorn,9,17,71,West Coast,11,11,77,M.C.G.,VIC
15528,15528,29-Jun-2019,R15,Sydney,14,9,93,Gold Coast,7,9,51,S.C.G.,NSW
15529,15529,29-Jun-2019,R15,Collingwood,5,7,37,North Melbourne,11,15,81,Docklands,VIC
15530,15530,29-Jun-2019,R15,Port Adelaide,5,11,41,Western Bulldogs,10,6,66,Adelaide Oval,SA
15531,15531,30-Jun-2019,R15,St Kilda,10,10,70,Richmond,16,7,103,Docklands,VIC
15532,15532,30-Jun-2019,R15,Brisbane Lions,15,17,107,Melbourne,11,8,74,Gabba,QLD
15533,15533,30-Jun-2019,R15,Fremantle,11,9,75,Carlton,11,13,79,Perth Stadium,WA


## 1. HVAR & AGGR

In [11]:
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2'] 

## 2. YEAR & MONTH

In [12]:
import time
import datetime

In [13]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

## 3. ABBREVIATED CLUB NAMES

In [14]:
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [16]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [17]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [18]:
trans['t1'].value_counts().sum()

15533

# 4. FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [19]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [20]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [21]:
trans["team2"].value_counts()

Collingwood         1281
Carlton             1259
Essendon            1229
Geelong             1226
Melbourne           1198
St Kilda            1196
Richmond            1127
North Melbourne      993
Hawthorn             987
Western Bulldogs     982
Fitzroy              960
South Melbourne      788
Sydney               438
West Coast           383
Brisbane             366
Adelaide             332
Fremantle            280
Port Adelaide        261
Gold Coast            95
GWS Giants            89
University            63
Name: team2, dtype: int64

In [22]:
trans["venue"].value_counts().tail(15)

Cazaly's Stadium       8
Yarraville Oval        7
Traeger Park           5
North Hobart           5
Eureka Stadium         4
Jiangwan Stadium       3
Wellington             3
Olympic Park           3
Brisbane Exhibition    1
Blacktown              1
Albury                 1
Riverway Stadium       1
Euroa                  1
Bruce Stadium          1
Yallourn               1
Name: venue, dtype: int64

# 5. ALL DRAWS (BOOLEAN)

In [23]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [24]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [25]:
trans['draw_rt'].value_counts()

0    15375
1      158
Name: draw_rt, dtype: int64

### The ET draws

In [26]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [27]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15531
1        2
Name: etime_draw, dtype: int64

In [28]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15373
1      160
Name: draws_all, dtype: int64

In [29]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10795,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13204,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


# 6. GAME CATEGORY

In [32]:
finals = ['EF','QF','SF','PF','GF']

In [33]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [34]:
trans["type"] = trans["round"].apply(game_cat)

In [35]:
trans['type'].value_counts()

reg_season    14878
final           655
Name: type, dtype: int64

In [36]:
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all,type
15530,15529,2019-06-29,R15,Collingwood,5,7,37,North Melbourne,11,15,...,-44,118,2019,6,CO,NM,0,0,0,reg_season
15531,15530,2019-06-29,R15,Port Adelaide,5,11,41,Western Bulldogs,10,6,...,-25,107,2019,6,PA,WB,0,0,0,reg_season
15532,15531,2019-06-30,R15,St Kilda,10,10,70,Richmond,16,7,...,-33,173,2019,6,SK,RI,0,0,0,reg_season
15533,15532,2019-06-30,R15,Brisbane,15,17,107,Melbourne,11,8,...,33,181,2019,6,BR,ME,0,0,0,reg_season
15534,15533,2019-06-30,R15,Fremantle,11,9,75,Carlton,11,13,...,-4,154,2019,6,FR,CA,0,0,0,reg_season


## 7. GROUPING COLUMNS

In [37]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [38]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [39]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [40]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

## 8. ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [41]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [42]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [43]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

## Export wide format results since 1897

In [44]:
trans.to_csv('wide_afl_run.csv')

# PAGEBREAK

# Convert data frame from wide to long format - each game appears twice (once for each team)

## 9. Reduce and re-order the data frame

In [45]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4


### Create two new columns with values 1 and 0 for the two subsets

In [46]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/stevegabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
tracker.head()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
2,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,0
3,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,0
4,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,0
5,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1,0
6,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1,0


### List for dropping 

In [48]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [49]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [50]:
left_team.shape

(15533, 10)

In [51]:
right_team.shape

(15533, 10)

### Rename columns in each dframe 

In [52]:
left_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1


In [53]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']

In [54]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1


In [55]:
right_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_2
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,0
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,0
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,0
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,0
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,0


In [57]:
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

In [58]:
right_team.tail()

,year,opp_team,op_pts,team,tm_pts,venue,op,tm,hvar,lr
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,0
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,0
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,0
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,0
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,0


### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [59]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15530,2019,North Melbourne,81,Collingwood,37,Docklands,NM,CO,-44,0
15531,2019,Western Bulldogs,66,Port Adelaide,41,Adelaide Oval,WB,PA,-25,0
15532,2019,Richmond,103,St Kilda,70,Docklands,RI,SK,-33,0
15533,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0
15534,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0


In [60]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15530,2019,Collingwood,37,North Melbourne,81,Docklands,CO,NM,-44,1
15531,2019,Port Adelaide,41,Western Bulldogs,66,Adelaide Oval,PA,WB,-25,1
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1


## CONCATENATION

### Now very easily, can concatenate the two dframes

In [61]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
2,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
3,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
4,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
5,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
6,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


In [62]:
merge_ft.shape

(31066, 10)

In [63]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15530,2019,North Melbourne,81,Collingwood,37,Docklands,NM,CO,-44,0
15531,2019,Western Bulldogs,66,Port Adelaide,41,Adelaide Oval,WB,PA,-25,0
15532,2019,Richmond,103,St Kilda,70,Docklands,RI,SK,-33,0
15533,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0
15534,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0


In [64]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
2,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
3,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
4,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
5,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
6,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
6,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


## Create team result column

In [65]:
def neg_hvar(hvar):
        return -1*hvar

In [66]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [67]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,33
15533,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0,-33
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,-33
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,4
15534,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0,4


In [68]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [69]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [70]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [71]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15532,2019,St Kilda,70,Richmond,103,Docklands,SK,RI,-33,1,33,1,-33
15533,2019,Melbourne,74,Brisbane,107,Gabba,ME,BR,33,0,-33,-1,-33
15533,2019,Brisbane,107,Melbourne,74,Gabba,BR,ME,33,1,-33,1,33
15534,2019,Fremantle,75,Carlton,79,Perth Stadium,FR,CA,-4,1,4,1,-4
15534,2019,Carlton,79,Fremantle,75,Perth Stadium,CA,FR,-4,0,4,-1,4


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [72]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [73]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
2,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
2,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
3,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
3,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
4,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


## Export as csv

In [74]:
merge_ft.to_csv('running_2019_R15.csv')